# Machine Learning and Comic Book Characters
Ariana Olson, Emma Price

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import thinkstats2
import thinkplot

import sklearn.linear_model as linear_model
import sklearn.metrics as metrics

import keras.models
import keras.layers

## Explore the data